In [4]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text
from sqlalchemy import create_engine, func
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import hvplot.pandas
# Create an engine
engine = create_engine('sqlite:///opt_3_db.db')

# Read the CSV file into a DataFrame
df = pd.read_csv('Resources/opt_3.csv')

# Load the DataFrame into the database
df.to_sql('opt_3_data', con=engine, if_exists='replace', index=False)
Session = sessionmaker(bind=engine)
session = Session()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [5]:
query_1 = 'SELECT * FROM opt_3_data'

In [6]:
result = session.execute(text(query_1))

df_result= pd.DataFrame(result)
df_result.head()

,Index,Year,Month,State,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count,Median Listing Price,Price Increased Count,Price Reduced Count
0,0,2024.0,10.0,in,45.0,54.0,14.0,146.0,2102.0,363246.0,61.0,277500.0,0.0,14.0
1,1,2024.0,10.0,ga,25.0,60.0,4.0,139.0,1729.0,295232.0,26.0,259900.0,0.0,4.0
2,2,2024.0,10.0,va,18.0,28.0,8.0,169.0,1764.0,379463.0,25.0,546175.0,20.0,462.0
3,3,2024.0,10.0,ga,9.0,46.0,4.0,183.0,1925.0,618415.0,11.0,313950.0,0.0,8.0
4,4,2024.0,10.0,tx,623.0,67.0,160.0,155.0,1837.0,339294.0,850.0,671725.0,212.0,794.0


In [7]:
#Scaling Data
# Scaling the numeric columns
data_scaled = StandardScaler().fit_transform(df_result[["Active Listing Count","Median Days on Market","New Listing Count","Median List Price Per Sqft",
                                                        "Median Sqft","Avg Listing Price","Total Listing Count", "Median Listing Price", 
                                                        "Price Increased Count", "Price Reduced Count"]])

# Creating a DataFrame with with the scaled data
df_transformed = pd.DataFrame(data_scaled, columns=["Active Listing Count","Median Days on Market","New Listing Count","Median List Price Per Sqft",
                                                        "Median Sqft","Avg Listing Price","Total Listing Count", "Median Listing Price", 
                                                        "Price Increased Count", "Price Reduced Count"])

# Display sample data
df_transformed.head()

,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count,Median Listing Price,Price Increased Count,Price Reduced Count
0,-0.340530,-0.591155,-0.341352,0.053791,0.549356,-0.009965,-0.339564,-0.098480,-0.259439,-0.276710
1,-0.361033,-0.405071,-0.363447,-0.014468,-0.420277,-0.201677,-0.364025,-0.177867,-0.259439,-0.312173
2,-0.368209,-1.397520,-0.354609,0.278072,-0.329293,0.035746,-0.364724,1.113402,0.237156,1.312027
3,-0.377435,-0.839268,-0.363447,0.414591,0.089235,0.709283,-0.374508,0.065931,-0.259439,-0.297988
4,0.252013,-0.187973,-0.018768,0.141553,-0.139525,-0.077479,0.211848,1.679706,5.004470,2.489395


In [8]:
state_encoded = pd.get_dummies(df_result["State"], columns=['State'])
state_encoded

,ak,al,ar,az,ca,co,ct,dc,de,fl,ga,hi,ia,id,il,in,ks,ky,la,ma,md,me,mi,mn,mo,ms,mt,nc,nd,ne,nh,nj,nm,nv,ny,oh,ok,or,pa,ri,sc,sd,tn,tx,ut,va,vt,wa,wi,wv,wy
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144671,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
144672,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
144673,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
144674,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [9]:
date_df = pd.DataFrame({
    "Year": df_result["Year"],
    "Month": df_result["Month"]
})
date_df

,Year,Month
0,2024.0,10.0
1,2024.0,10.0
2,2024.0,10.0
3,2024.0,10.0
4,2024.0,10.0
...,...,...
144671,2016.0,7.0
144672,2016.0,7.0
144673,2016.0,7.0
144674,2016.0,7.0


In [18]:
date_df["Year"] = date_df["Year"].astype(int)
date_df["Month"] = date_df["Month"].astype(int)

date_df

,Year,Month
0,2024,10
1,2024,10
2,2024,10
3,2024,10
4,2024,10
...,...,...
144671,2016,7
144672,2016,7
144673,2016,7
144674,2016,7


In [19]:
#Concatinating 
combined_df = pd.concat([date_df, df_transformed, state_encoded], axis=1)
combined_df.head()

,Year,Month,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count,Median Listing Price,Price Increased Count,Price Reduced Count,ak,al,ar,az,ca,co,ct,dc,de,fl,ga,hi,ia,id,il,in,ks,ky,la,ma,md,me,mi,mn,mo,ms,mt,nc,nd,ne,nh,nj,nm,nv,ny,oh,ok,or,pa,ri,sc,sd,tn,tx,ut,va,vt,wa,wi,wv,wy
0,2024,10,-0.340530,-0.591155,-0.341352,0.053791,0.549356,-0.009965,-0.339564,-0.098480,-0.259439,-0.276710,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2024,10,-0.361033,-0.405071,-0.363447,-0.014468,-0.420277,-0.201677,-0.364025,-0.177867,-0.259439,-0.312173,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2024,10,-0.368209,-1.397520,-0.354609,0.278072,-0.329293,0.035746,-0.364724,1.113402,0.237156,1.312027,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,2024,10,-0.377435,-0.839268,-0.363447,0.414591,0.089235,0.709283,-0.374508,0.065931,-0.259439,-0.297988,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,2024,10,0.252013,-0.187973,-0.018768,0.141553,-0.139525,-0.077479,0.211848,1.679706,5.004470,2.489395,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [20]:
X = combined_df.drop(columns=['Avg Listing Price'])
y = combined_df['Avg Listing Price'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(115740, 62) (28936, 62) (115740,) (28936,)


In [21]:
# Create a model with scikit-learn
model = LinearRegression()

# Fit the data into the model
model.fit(X, y)

LinearRegression()

In [22]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [23]:
# Create a copy of the original data
df_predicted = combined_df.copy()

# Add a column with the predicted salary values
df_predicted["Avg_median_predicted"] = predicted_y_values

# Display sample data
df_predicted.head()

,Year,Month,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count,Median Listing Price,Price Increased Count,Price Reduced Count,ak,al,ar,az,ca,co,ct,dc,de,fl,ga,hi,ia,id,il,in,ks,ky,la,ma,md,me,mi,mn,mo,ms,mt,nc,nd,ne,nh,nj,nm,nv,ny,oh,ok,or,pa,ri,sc,sd,tn,tx,ut,va,vt,wa,wi,wv,wy,Avg_median_predicted
0,2024,10,-0.340530,-0.591155,-0.341352,0.053791,0.549356,-0.009965,-0.339564,-0.098480,-0.259439,-0.276710,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.115528
1,2024,10,-0.361033,-0.405071,-0.363447,-0.014468,-0.420277,-0.201677,-0.364025,-0.177867,-0.259439,-0.312173,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.138760
2,2024,10,-0.368209,-1.397520,-0.354609,0.278072,-0.329293,0.035746,-0.364724,1.113402,0.237156,1.312027,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0.068600
3,2024,10,-0.377435,-0.839268,-0.363447,0.414591,0.089235,0.709283,-0.374508,0.065931,-0.259439,-0.297988,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.283466
4,2024,10,0.252013,-0.187973,-0.018768,0.141553,-0.139525,-0.077479,0.211848,1.679706,5.004470,2.489395,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0.047161


In [24]:
from sklearn.metrics import r2_score

In [25]:
r2_score( combined_df['Avg Listing Price'] ,predicted_y_values)

0.7436066522554001